In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
from config import api_key

from census import Census
from us import states

In [ ]:
#TK Code

In [ ]:
print ("I made changes to the file")
print("Added new print stmt")
print("last stmt")

In [ ]:
#TK code ends

In [1]:
#Adam code is awesome

In [ ]:
c = Census(api_key, year=2016)
raw_data = c.acs5.get(('NAME', 'B19013_001E', 'B19301_001E', 'B23025_003E',
                       'B23025_004E', 'B23025_005E', 'B01003_001E', "B01002_001E"), {'for':'state:*'})
census_df = pd.DataFrame(raw_data)
census_df = census_df.rename(columns = {'state': 'State#',
                          'NAME': 'State', 
                          'B01003_001E':'Total_population',
                          'B23025_003E':'Employable_civilians',
                          'B23025_004E':'Employed_civilians', 
                          'B23025_005E':'Unemployed_civilians', 
                          'B19013_001E':'Income',
                          'B19301_001E':'Income_per_capita',
                          "B01002_001E":'median age'
                          
    
})
census_df = census_df.sort_values('State#').reset_index(drop=True)
census_df.head()

In [ ]:
#Presidential file to dataframe called df
file = "Resources/1976-2016-president.tab"
df = pd.read_csv(file,sep='\t',header=(0))
#df.tail(40)

In [ ]:
party = df.groupby("party")
party = party["candidatevotes"].sum()
party = pd.DataFrame(party.sort_values(ascending=False))
#party.head(40)

In [ ]:
party_state = df.groupby(["party", "state"])
#party_state.sum()

In [ ]:
#list just republican/democrat
parties = list(df["party"].unique())
parties
#would consider conservative, republican, and liberal party democrat. need to combine. 
combined_dem_rep = df.replace("conservative","republican")
combined_dem_rep = combined_dem_rep.replace("liberal party", "democrat")
combined_dem_rep = combined_dem_rep.replace("democratic-farmer-labor","democrat")
dems = combined_dem_rep.loc[combined_dem_rep["party"] == "democrat"]
repubs = combined_dem_rep.loc[combined_dem_rep["party"] == "republican"]
dems_repubs = dems.append(repubs)
state_count = dems_repubs.groupby(["party", "state"])
state_count = pd.DataFrame(state_count[["candidatevotes", "totalvotes"]].sum())
#state_count

In [ ]:
dem_votes_by_state = dems[["state","candidatevotes","totalvotes"]]
dem_votes_by_state = dem_votes_by_state.groupby("state")
dem_votes_by_state = dem_votes_by_state.sum()
len(dem_votes_by_state)

repubs_votes_by_state = repubs[["state","candidatevotes","totalvotes"]]
repubs_votes_by_state = repubs_votes_by_state.groupby("state")
repubs_votes_by_state = repubs_votes_by_state.sum()
#repubs_votes_by_state

dems_merged_repubs = pd.merge(dem_votes_by_state, repubs_votes_by_state, on="state", how="outer")
dems_merged_repubs = dems_merged_repubs.rename(columns = {"candidatevotes_x":"democrat votes",
                                                         "candidatevotes_y": "republican votes"})
dems_merged_repubs = dems_merged_repubs[['democrat votes', 'republican votes']]
dems_merged_repubs["Difference%ofTotal"] = (dems_merged_repubs["democrat votes"] - dems_merged_repubs["republican votes"])/(dems_merged_repubs["democrat votes"]+dems_merged_repubs["republican votes"])
dems_merged_repubs = dems_merged_repubs.sort_values("Difference%ofTotal", ascending=False)
#dems_merged_repubs



In [ ]:
plots = list(dems_merged_repubs["Difference%ofTotal"])
x_axis = list(dems_merged_repubs.index)
colors = ["blue",
          "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",]
plt.bar(x_axis, plots, color=colors, alpha=0.5, align="center")
plt.tight_layout()
plt.xlabel("States")
plt.ylabel("democrat - republican votes / total votes")
plt.title("Difference from total democrat votes as % of total votes 1976 - 2016")
plt.xticks(rotation=90)
plt.show()


In [ ]:
# 1980 - 1996
dems_repubs_1980_forward = dems_repubs.loc[dems_repubs["year"]>=1980]
dems_repubs_1980_1996 = dems_repubs_1980_forward.loc[dems_repubs_1980_forward["year"]<=1996]
#dems_repubs_1980_1996
dems_repubs_2000_2016 = dems_repubs_1980_forward.loc[dems_repubs_1980_forward["year"]>= 2000]
#dems_repubs_2000_2016

In [ ]:
# plot for 1980 - 1996 

early_dems = dems_repubs_1980_1996.loc[dems_repubs_1980_1996["party"]=="democrat"]
early_dems = early_dems.groupby("state")
early_dems = pd.DataFrame(early_dems["candidatevotes"].sum())
#early_dems

early_repubs = dems_repubs_1980_1996.loc[dems_repubs_1980_1996["party"]=="republican"]
early_repubs = early_repubs.groupby("state")
early_repubs = pd.DataFrame(early_repubs["candidatevotes"].sum())
#early_repubs

early_merge = pd.merge(early_dems, early_repubs, how="outer", on="state")
early_merge = early_merge.rename(columns = {"candidatevotes_x": "democrat votes 1980-1996",
                                           "candidatevotes_y": "republican votes 1980-1996"})
early_merge["Difference%ofTotal"] = (early_merge["democrat votes 1980-1996"] - early_merge["republican votes 1980-1996"])/(early_merge["democrat votes 1980-1996"]+early_merge["republican votes 1980-1996"])
early_merge

late_dems = dems_repubs_2000_2016.loc[dems_repubs_2000_2016["party"]=="democrat"]
late_dems = late_dems.groupby("state")
late_dems = pd.DataFrame(late_dems["candidatevotes"].sum())
#late_dems

late_repubs = dems_repubs_2000_2016.loc[dems_repubs_2000_2016["party"]=="republican"]
late_repubs = late_repubs.groupby("state")
late_repubs = pd.DataFrame(late_repubs["candidatevotes"].sum())
#late_repubs

late_merge = pd.merge(late_dems, late_repubs, how="outer", on="state")
late_merge = late_merge.rename(columns = {"candidatevotes_x": "democrat votes 2000-2016",
                                         "candidatevotes_y": "republican votes 2000-2016"})
late_merge["Difference%ofTotal_late"] = (late_merge["democrat votes 2000-2016"] - late_merge["republican votes 2000-2016"])/(late_merge["democrat votes 2000-2016"]+late_merge["republican votes 2000-2016"])
late_merge

early_late_merge = pd.merge(early_merge, late_merge, how="outer", on="state")
early_late_merge["change"] = early_late_merge["Difference%ofTotal_late"] - early_late_merge["Difference%ofTotal"]
early_late_merge = early_late_merge.sort_values("change", ascending=False)
#negative indicates change towards republican - positive indicates change towards democrat

#early_late_merge


In [ ]:
plots = list(early_late_merge["change"])
x_axis = list(early_late_merge.index)
colors = ["blue",
          "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "blue",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",
         "red",]
plt.bar(x_axis, plots, color=colors, alpha=0.5, align="center")
plt.tight_layout()
plt.xlabel("States")
plt.ylabel("shift in favored party - positive is shift towards dem, negative shift towards repubs")
plt.title("Shift in party preference by states")
plt.xticks(rotation=90)
plt.show()


In [ ]:
early_late_merge.loc[early_late_merge["change"] > 0, "shift"] = "Democrat"
early_late_merge.loc[early_late_merge["change"] < 0, "shift"] = "Republican"
early_late_merge["absolute value"] = early_late_merge["change"].abs()
early_late_merge
#early_late_merge

In [ ]:
dems_repubs_minnesota = dems_repubs.loc[dems_repubs["state"] == "Minnesota"]
dems_repubs_minnesota = dems_repubs_minnesota.groupby("party").sum()
dems_repubs_minnesota = dems_repubs_minnesota[["candidatevotes","totalvotes"]]
#dems_repubs_minnesota

In [ ]:
#list(df["party"].unique())

In [ ]:
#just curious about write in candidates
just_write_in = df.loc[df["writein"] == True]
#just_write_in

In [ ]:
write_in_by_state = just_write_in.groupby("state")
write_in_by_state = pd.DataFrame(write_in_by_state["candidatevotes"].sum())
sorted_write_in_by_state = write_in_by_state["candidatevotes"].sort_values(ascending=False)
#sorted_write_in_by_state

In [ ]:
candidate = df.groupby("candidate")
temp = candidate.sum()
temp["% of totalvotes"] = temp["candidatevotes"]/temp["totalvotes"]
temp = pd.DataFrame(temp["% of totalvotes"].sort_values(ascending=False))
#temp.head(25)

In [ ]:
year_2016 = df.loc[df["year"]==2016]

by_candidate_2016 = year_2016.groupby("candidate")
total_votes = by_candidate_2016["candidatevotes"].sum().sort_values(ascending=False)
#total_votes

In [ ]:
year_2012 = df.loc[df["year"]==2012]

by_candidate_2012 = year_2012.groupby("candidate")
total_votes_2012 = by_candidate_2012["candidatevotes"].sum().sort_values(ascending=False)
#total_votes_2012

In [ ]:
#Adam code ends

In [ ]:
#Ryan code

In [ ]:
from census import Census
from us import states

c = Census(api_key, year=2018)

In [ ]:
* As only civilians are legel to vote, it would make sense to only pull the civilian employment data.
* The labor force is the sum of employed and unemployed persons
* Columns data to pull: 
* Income variables: 
    "income": B19013_001E, "income_per_capita": B19301_001E
* Employment variables:
    "employable civilians": B23025_003E, "employed civilians": B23025_004E, "unemployed civilians": B23025_005E
* Population variables: 
    "Total population": B01003_001E
* Poverty variables:
    "poverty people": "B17001_002E"
* Age variables:
    "median age": "B01002_001E" 

In [ ]:
raw_data = c.acs5.get(('NAME', 'B19013_001E', 'B19301_001E', 'B23025_003E',
                       'B23025_004E', 'B23025_005E', 'B01003_001E', "B01002_001E"), {'for':'state:*'})
df = pd.DataFrame(raw_data)
df = df.rename(columns = {'state': 'State#',
                          'NAME': 'State', 
                          'B01003_001E':'Total_population',
                          'B23025_003E':'Employable_civilians',
                          'B23025_004E':'Employed_civilians', 
                          'B23025_005E':'Unemployed_civilians', 
                          'B19013_001E':'Income',
                          'B19301_001E':'Income_per_capita',
                          "B01002_001E":'median age'
                          
    
})
df = df.sort_values('State#').reset_index(drop=True)
df

In [ ]:
#Ryan code ends

In [ ]:
#Connor code

In [ ]:
print("I made a change too!")

In [ ]:
#Connor code ends